<span style="color:lightgreen; font-size:30px">**PG303 - Análisis geoespacial**</span>
***
<span style="color:gold; font-size:30px">**Rioxarray**</span>
***

<span style="font-size:20px"> **Autor: Kevin Alexander Gómez** </span>

<span style="font-size:16px"> **Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)** </span>

***

Bienvenido al curso PG303 - Análisis geoespacial!!!

Vamos a revisar aplicaciones de <span style="color:gold">análisis geoespacial</span> usando código en Python.\
Es necesario que tengas un conocimiento previo en programación con Python, estadística y sistemas de información geográfica.

<span style="color:lightgreen"> Este notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología. </span>

En el siguiente índice, encontrarás los temas que componen este notebook:

## **Índice**
***
- [¿Qué es Rioxarray?](#parte-1)
- [Abriendo un raster en Python](#parte-2)
- [Estadística descriptiva de un raster](#parte-3)
- [Convirtiendo una imagen raster a DataFrame](#parte-4)

***

Antes de empezar tu camino en programación geológica...\
Recuerda que puedes ejecutar un bloque de código usando `Shift` + `Enter`:

In [ ]:
2 + 2

Si por error haces doble clic sobre un bloque de texto (como el que estás leyendo ahora mismo), puedes arreglarlo usando también `Shift` + `Enter`.

***

<a id="parte-1"></a>

### <span style="color:lightgreen">**¿Qué es Rioxarray?**</span>
***

<span style="color:gold">Rioxarray</span> es una librería de Python utilizada en el análisis de datos geoespaciales.\
Basándose en la librería <span style="color:gold">xarray</span>, proporciona una estructura de datos etiquetada y multidimensional llamada `DataArray`. 

Rioxarray extiende las capacidades de xarray al agregar funcionalidades específicas para trabajar con datos geoespaciales, como la capacidad de leer y escribir archivos raster en diferentes formatos, realizar operaciones espaciales como recortar y re-muestrear datos, y calcular índices y estadísticas comunes utilizadas en el análisis geoespacial.

<span style="color:#43c6ac">Está librería ha sido diseñada para simplificar el análisis y la manipulación de datos geoespaciales multidimensionales, como imágenes satelitales, datos de sensores remotos y conjuntos de datos climáticos.</span>

Algunas de las características clave de rioxarray incluyen también la compatibilidad con una amplia gama de formatos de archivo geoespaciales, la capacidad de reproyectar y transformar datos espaciales, el soporte para operaciones algebraicas y lógicas en datos georreferenciados, y la integración con otras bibliotecas de análisis de datos populares, como NumPy y Pandas.

<img src="resources/xarray_graph.png" style="background-color:white" width=90%>

***

<a id="parte-2"></a>

### <span style="color:lightgreen">**Abriendo un raster en Python**</span>
***

Para abrir un raster dentro de Python usando rioxarray, empezaremos importando las librerías necesarias:

In [ ]:
import rioxarray as riox

Ahora, colocaremos la ruta de nuestro raster y usaremos la función `open_rasterio` para abrirlo:
> Usaremos el raster llamado `peru_60.tiff`, que contiene un modelo digital de elevación del Perú en baja resolución (60 arco-segundos).

In [ ]:
# Ruta del archivo raster
ruta = "files/peru_60.tiff"

# Abrir el raster utilizando rioxarray
raster = riox.open_rasterio(ruta)

# Mostrar información del raster
print(raster)

Podemos leer algunas de las características principales de este raster como:
- La cantidad de bandas o canales: 1 para la elevación (en una imagen a color el RGB presenta de 3 a 4 bandas)
- La extensión del raster en píxeles: 1110 de ancho x 840 de largo
- La cantidad total de píxeles: 932400
- Las coordenadas de los píxeles en el espacio, que se encuentran en coordenadas geográficas (WGS84)
- Y las estadísticas de los valores de elevación, como la altura máxima (5427) y mínima (-4779) registradas con esta resolución

Ahora, seleccionaremos la matriz con los valores de elevación, para esto usaremos el atributo `values` y como solo tenemos una banda, colocaremos el primer índice de posición 0 para seleccionar esta matriz:

In [ ]:
# Acceder a los datos del raster (banda 0)
raster.values[0]

Usando el atributo `coords`, podemos mirar el contenido de las coordenadas espaciales del raster:

In [ ]:
raster.coords

Podemos obtener el sistema de coordenadas del raster usando el atributo `rio.crs`:

In [ ]:
raster.rio.crs

Este código `EPSG:4326` representa el sistema de coordenadas geográfico WGS84, que usa unidades de grados sexagesimales.
***

<a id="parte-3"></a>

### <span style="color:lightgreen">**Estadística descriptiva de un raster**</span>
***

Usando la matriz de valores de elevación, crearemos algunos gráficos para describir su contenido.

Empezaremos mostrando nuevamente los valores de elevación y los almacenaremos en la variable `DEM`:

In [ ]:
DEM = raster.values[0]
DEM

Ahora, graficaremos un histograma del valor de elevación: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(10, 5))

# Histograma
ax.hist(DEM.flatten(), edgecolor="black", color="lightgreen", bins=100, alpha=0.8)

# Texto
ax.set_title("Histograma de elevación", fontsize=18)
ax.set_xlabel("Elevación (m)", fontsize=15)
ax.set_ylabel("Frecuencia", fontsize=15)

# Grilla
ax.grid(lw=1)
ax.set_axisbelow(True)

plt.show()

Notamos 3 modas en el histograma:
- La primera moda tiene un nivel de elevación cercano a los -5000 metros, que corresponde al fondo marino.
- La segunda moda tiene un nivel de elevación cercano a los 100 metros, que corresponde a la altura cerca del nivel del mar.
- Por último, la tercera moda tiene un nivel de elevación cercano a los 4000 metros, y puede corresponder a la cordillera de los Andes.

Para verificar esto, visualizaremos el raster con la función `imshow` de Matplotlib:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(9, 6))

# Imagen
ax.imshow(DEM, cmap="terrain")

# Texto
ax.set_title("Modelo digital de elevación", fontsize=15)

# Esconder los ejes
fig.set_facecolor("white")
ax.set_xticks([])
ax.set_xticklabels([])
ax.set_yticks([])
ax.set_yticklabels([])

plt.show()

La imagen revela la distribución geoespacial de elevaciones en el Perú, los valores oscuros pertenecen al fondo marino y los colores claros a las zonas de la superficie continental.

- Nótese una línea curva que va desde la esquina superior izquierda hacia la esquina inferior derecha de la imagen, esta línea oscura gruesa marca la zona conocida como Fosa Perú - Chile.
- Las zonas con una tonalidad blanquecina corresponden a la cordillera de los Andes, mientras que las regiones con una tonalidad más grisácea hacia el Este de la cordillera corresponde a la selva del Amazonas.
- Algunos valores de elevación cercanos al nivel del mar pero negativos corresponden a lo que se conoce como la plataforma continental, ubicado frente a la costa.


En este ejemplo, no es posible distinguir la línea de costa en el modelo de elevación, por lo tanto, usaremos un filtro que seleccione las áreas con una elevación positiva:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(12, 8))

# Imagen filtro
ax.imshow(np.where(DEM > 0, DEM, np.nan), cmap="terrain")

# Texto
ax.set_title("Modelo digital de elevación", fontsize=18)

# Esconder los ejes
fig.set_facecolor("white")
ax.set_xticks([])
ax.set_xticklabels([])
ax.set_yticks([])
ax.set_yticklabels([])

plt.show()

***

<a id="parte-4"></a>

### <span style="color:lightgreen">**Convirtiendo una imagen raster a DataFrame**</span>
***

¿Es posible transformar una imagen raster en filas y columnas dentro de un DataFrame de Pandas?, la respuesta es Sí.

Para lograr esto, empezaremos realizando lo siguiente:
- Usaremos los métodos `squeeze` y `drop` para eliminar algunos detalles que no serán necesarios incluir en el DataFrame.
- Almacenaremos el resultado en la variable `data_array`.
- También debemos colocarle un nombre a nuestro DataArray, por ejemplo, usaremos la letra `Z`.

In [ ]:
data_array = raster.squeeze().drop(["band", "spatial_ref"])
data_array.name = "Z"
data_array

Y ahora, convertimos el DataArray en un DataFrame usando el método `to_dataframe`, y resetearemos el índice con `reset_index`, almacenando el resultado en la variable `df`:

In [ ]:
df = data_array.to_dataframe().reset_index()
df.head()

Notamos que el DataFrame tiene 3 columnas:
- `x` e `y`: corresponden a la longitud y latitud en grados sexagesimales.
- `Z`: es la elevación del raster, en metros.

Renombraremos las columnas usando el método `rename`:

In [ ]:
df.rename(columns={"x": "LON", "y": "LAT", "Z": "ELEV"}, inplace=True)
df.head()

Ahora, ya tenemos un DataFrame con la información de la imagen raster, listo para ser utilizado en diferentes actividades.

Algunas de las principales ventajas de usar una imagen raster convertida en DataFrame son:
- Facilita la manipulación y procesamiento de datos geoespaciales, como la selección e indexación
- Permite tratar los valores erróneos o faltantes en el raster
- Permite integrar el raster a un flujo de trabajo más grande y complejo, relacionado al análisis geoespacial

Sin embargo, es importante tener en cuenta que al pasar una imagen raster a un DataFrame de Pandas, la estructura de datos resultante puede requerir más memoria y potencialmente afectar el rendimiento, especialmente si la imagen es muy grande.

Además, algunas operaciones que son eficientes en formato raster, como el cálculo de estadísticas zonales, pueden volverse más complejas y menos eficientes en un DataFrame de Pandas.

***